<a href="https://colab.research.google.com/github/ishitananda3/Data-analysis-of-iris-flower-set/blob/main/LLM_Company_Performance_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.44.0
    Uninstalling openai-1.44.0:
      Successfully uninstalled openai-1.44.0


In [ ]:
# Imports and API Setup

import openai
import datetime
import json

# Set your API key here
openai.api_key = 'your_open_api_key'


In [ ]:
# Date Handling Functions

def get_default_dates():
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=365)
    return start_date, end_date

def convert_to_iso(date_str):
    # Ensure the date is in the correct format (YYYY-MM-DD)
    return date_str.isoformat()


In [ ]:
#Querying OpenAI for Information Extraction

def extract_information_from_query(query):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo", # or "gpt-3.5-turbo" depending on your access
        prompt=f"Extract the following information from this query: {query}. Information: Company name, metric (e.g., revenue, profit), start date, end date.",
        max_tokens=150
    )

    # Process the GPT output here
    return response['choices'][0]['text']


In [ ]:
# Handling Variations in Queries

def parse_llm_response(response_text):
    # Parse the GPT response for necessary fields
    company_name = None
    metric = None
    start_date = None
    end_date = None

    # Use simple text parsing for now (could improve with regex or NLP tools)
    # Ideally, GPT's output is clean and structured like:
    # Company: Amazon
    # Metric: revenue
    # Start Date: 2023-01-01
    # End Date: 2023-12-31

    for line in response_text.split("\n"):
        if "Company" in line:
            company_name = line.split(":")[1].strip()
        elif "Metric" in line:
            metric = line.split(":")[1].strip()
        elif "Start Date" in line:
            start_date = line.split(":")[1].strip()
        elif "End Date" in line:
            end_date = line.split(":")[1].strip()

    return company_name, metric, start_date, end_date


In [ ]:
#Handling Missing Dates and Formatting

def handle_missing_dates(start_date, end_date):
    if not start_date or not end_date:
        default_start, default_end = get_default_dates()
        start_date = start_date or convert_to_iso(default_start)
        end_date = end_date or convert_to_iso(default_end)
    return start_date, end_date


In [ ]:
#Converting the Extracted Information to JSON

def format_as_json(company_name, metric, start_date, end_date):
    data = {
        "entity": company_name,
        "parameter": metric,
        "start_date": start_date,
        "end_date": end_date
    }
    return json.dumps(data, indent=4)


In [ ]:
# Main Function to Process User Query

def process_query(query):
    # Call GPT to get the response
    response = extract_information_from_query(query)

    # Parse the response to extract the information
    company_name, metric, start_date, end_date = parse_llm_response(response)

    # Handle missing dates (defaults to past year)
    start_date, end_date = handle_missing_dates(start_date, end_date)

    # Format the information as JSON
    output_json = format_as_json(company_name, metric, start_date, end_date)
    return output_json


In [ ]:
query = "Can you show me the revenue of Amazon from the last quarter?"
print(process_query(query))